In [1]:
!pip install chess


In [1]:
import chess
import chess.svg
import chess.engine
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from model import KnightVisionNN, KingsVisionNN2
from preprocess import fen_to_vector, encode_moves, TopMovesDataset
from tqdm import tqdm
import matplotlib as plt

In [3]:
engine_path = "stockfish/stockfish-macos-x86-64-avx2"
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

In [2]:
data5000 = pd.read_csv("datasets/top_move5000.csv")
data5000.head()

,fen,top_moves
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,"['c7c5', 'e7e6', 'e7e5', 'c7c6', 'b8c6']"
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,"['b1c3', 'd2d4', 'g1f3', 'h2h3', 'c2c3']"
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,"['d7d5', 'a7a6', 'h7h6', 'b7b6', 'c7c6']"
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,"['b1d2', 'b1c3', 'e4e5', 'e4d5', 'f1b5']"
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,"['c7c5', 'f8e7', 'a7a6', 'd5e4', 'b8d7']"


In [3]:
all_moves = set()
for top_moves in data5000['top_moves']:
    moves = eval(top_moves) 
    all_moves.update(moves)
move_directory = {move: idx for idx, move in enumerate(all_moves)}

def encode_moves(top_moves, move_directory):
    return [move_directory[move] for move in top_moves]

data5000['EncodedMoves'] = data5000['top_moves'].apply(lambda moves: encode_moves(eval(moves), move_directory))
data5000.head()
len(move_directory)

1633

In [6]:
def get_top_moves(fen, top_x=5, depth=12):
    """
    Get the top X moves for a given FEN position using Stockfish.
    
    Parameters:
    fen (str): FEN string representing the chess position.
    top_x (int): Number of top moves to retrieve.
    depth (int): Depth of the search (higher means more accuracy but slower).
    
    Returns:
    list: List of top X moves in UCI format.
    """
    board = chess.Board(fen)
    result = engine.analyse(board, chess.engine.Limit(depth=depth), multipv=top_x)

    top_moves = []
    for entry in result:
        move = entry["pv"][0]  
        top_moves.append(move.uci()) 
    
    return top_moves

def create_fen_dataset_from_csv(csv_file, num_positions=200, top_x=5, depth=12):
    """
    Create a dataset of FEN positions and their top X moves from a CSV file.
    
    Parameters:
    csv_file (str): Path to the CSV file containing FEN positions and evaluations.
    num_positions (int): Number of positions to process.
    top_x (int): Number of top moves to retrieve.
    depth (int): Depth of the search.
    
    Returns:
    pd.DataFrame: DataFrame containing FEN positions and their top moves.
    """
    df = pd.read_csv(csv_file)
    
    fens = df[:num_positions]
    fens = fens['FEN'].tolist()
    
    selected_fens = fens[:num_positions]
    
    dataset = []
    for fen in tqdm(selected_fens, desc="Processing FENs"):
        try:
            top_moves = get_top_moves(fen, top_x=top_x, depth=depth)
            dataset.append({"fen": fen, "top_moves": top_moves})
        except Exception as e:
            print(f"Error processing FEN: {fen}, {e}")

    df_dataset = pd.DataFrame(dataset)
    return df_dataset

csv_file = "datasets/chess evaluations/chessData.csv"  # Path to your CSV file
chess_data = create_fen_dataset_from_csv(csv_file, num_positions=200, top_x=5, depth=15)

print(chess_data.head())
engine.quit()


Processing FENs:   6%|▋         | 13/200 [00:08<02:08,  1.45it/s]


KeyboardInterrupt: 

In [ ]:
output_csv_file = 'datasets/top_moves2'
chess_data.to_csv(output_csv_file, index=False)


In [ ]:
test1 = pd.read_csv('datasets/top_moves2')
test1.head()

In [4]:
testfen = test1['fen'].iloc[99]
board = chess.Board(testfen)
board

NameError: name 'test1' is not defined

In [19]:
data = data5000
all_moves = set()
for top_moves in data5000['top_moves']:
    moves = eval(top_moves)  
move_directory = {move: idx for idx, move in enumerate(all_moves)}
data["EncodedMoves"] = data["top_moves"].apply(lambda moves: encode_moves(eval(moves), move_directory))

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = KnightVisionNN(len(move_directory)).to(device)
net.load_state_dict(torch.load('./chess.pth'))
net.eval()
def predict_top_moves(data, move_directory):
    # Dataloader for tensor conversion
    dataset = TopMovesDataset(data[:10], move_directory)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
    
    reverse_move_directory = {idx: move for move, idx in move_directory.items()}
    
    predictions = []
    
    with torch.no_grad():
        for input_tensor, _ in dataloader:
            input_tensor = input_tensor.to(device)  
            logits = net(input_tensor)  
            probabilities = torch.sigmoid(logits)  
            top_probabilities, top_indices = torch.topk(probabilities, 5, dim=-1)
            top_moves_indices = top_indices.cpu().numpy().flatten()
            top_moves = [reverse_move_directory[idx] for idx in top_moves_indices]
            predictions.append(top_moves_indices)
    
    return predictions


predicted_top_moves = predict_top_moves(data5000, move_directory)
for i, top_moves in enumerate(predicted_top_moves[:10]):
    print(f"Top moves for position {i+1}: {top_moves}")


Top moves for position 1: [ 335 1317  489 1350  274]
Top moves for position 2: [1200 1120  478  932  944]
Top moves for position 3: [ 335 1317  489  274 1350]
Top moves for position 4: [1200 1120  932  478  944]
Top moves for position 5: [1317  335  489  274 1350]
Top moves for position 6: [1200 1120  932 1370 1042]
Top moves for position 7: [1317  335  489  274 1350]
Top moves for position 8: [1200 1120  932  478 1370]
Top moves for position 9: [1317  335  489  274 1350]
Top moves for position 10: [1200 1120  932  478 1370]


In [34]:
data[:10].head()

,fen,top_moves,EncodedMoves
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,"['c7c5', 'e7e6', 'e7e5', 'c7c6', 'b8c6']","[173, 281, 380, 552, 766]"
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,"['b1c3', 'd2d4', 'g1f3', 'h2h3', 'c2c3']","[1382, 1255, 1353, 38, 388]"
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,"['d7d5', 'a7a6', 'h7h6', 'b7b6', 'c7c6']","[336, 836, 854, 935, 552]"
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,"['b1d2', 'b1c3', 'e4e5', 'e4d5', 'f1b5']","[23, 1382, 1066, 596, 608]"
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,"['c7c5', 'f8e7', 'a7a6', 'd5e4', 'b8d7']","[173, 659, 836, 1325, 124]"


In [32]:
def decode_indices(indices, move_directory):
    inverse_move_directory = {v: k for k, v in move_directory.items()}
    return [inverse_move_directory.get(index, 'Unknown Move') for index in indices]

def test(model, data1, move_directory, device):
    model.eval()
    for idx, row in data1.iterrows():
        fen_position = row['fen']
        encoded_moves = row['top_moves']
    
        dataset = TopMovesDataset(data1, move_directory)
        dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
        
        with torch.no_grad():
            for input_tensor, _ in dataloader:
                logits = model(input_tensor)
                top_probabilities, top_indices = torch.topk(logits, 5, dim=-1)
        

        predicted_moves = decode_indices(top_indices.cpu().numpy().flatten(), move_directory)
        print(f"FEN: {fen_position}")
        print(f"Predicted moves: {predicted_moves}")
        print(f"Actual moves: {encoded_moves}")
        print("")
test(net, data[:5], move_directory, device)


FEN: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
Predicted moves: ['c3d3', 'f5h4', 'c8c2', 'c6d5', 'f7h5']
Actual moves: ['c7c5', 'e7e6', 'e7e5', 'c7c6', 'b8c6']

FEN: rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Predicted moves: ['c3d3', 'f5h4', 'c8c2', 'c6d5', 'f7h5']
Actual moves: ['b1c3', 'd2d4', 'g1f3', 'h2h3', 'c2c3']

FEN: rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBNR b KQkq - 0 2
Predicted moves: ['c3d3', 'f5h4', 'c8c2', 'c6d5', 'f7h5']
Actual moves: ['d7d5', 'a7a6', 'h7h6', 'b7b6', 'c7c6']

FEN: rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKBNR w KQkq - 0 3
Predicted moves: ['c3d3', 'f5h4', 'c8c2', 'c6d5', 'f7h5']
Actual moves: ['b1d2', 'b1c3', 'e4e5', 'e4d5', 'f1b5']

FEN: rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQKBNR b KQkq - 1 3
Predicted moves: ['c3d3', 'f5h4', 'c8c2', 'c6d5', 'f7h5']
Actual moves: ['c7c5', 'f8e7', 'a7a6', 'd5e4', 'b8d7']

